In [1]:
using Random

In [2]:
"""
D: the dimension.
Z: the coordination number.
"""
abstract type AbstractLattice{D, Z} end

struct Chain<:AbstractLattice{1, 2}
    length::Int
    Chain(length::Int) = new(length)
end

nsite(c::Chain) = c.length
neighbors(c::Chain, i::Int) = [mod(i-2, c.length)+1, mod(i, c.length)+1]
nsite(c::Chain) = c.length
nunitcells(c::Chain) = 1

nunitcells (generic function with 1 method)

In [13]:
const SSESpins = Matrix{Int16}
ssespins(nsite::Int, ntau::Int) = rand([-Int16(1), Int16(1)], nsite, ntau)
ntau(spins::SSESpins) = size(spins, 2)
nsite(spins::SSESpins) = size(spins, 1)

nsite (generic function with 2 methods)

In [14]:
abstract type AbstractModel{LT<:AbstractLattice} end

struct TFIModel{LT<:AbstractLattice} <: AbstractModel{LT}
    J0::Float64
    Γ::Float64
    temp::Float64
    lattice::LT
    
    function TFIModel(J0::Real, Γ::Real, temp::Real, lattice::LT) where LT<:AbstractLattice
        new{LT}(Float64(J0), Float64(Γ), Float64(temp), lattice)
    end
end

J_Trotter (generic function with 1 method)

In [16]:
N = 27
lt = Chain(N)
model = TFIModel(1.0, 0.5, 0.1, lt)
spins = ssespins(N, 200);

In [17]:
struct MCConfig
    ntherm::Int
    nmeas::Int
    seed::Int
end

chseed(mcc::MCConfig, seed::Int) = MCConfig(mcc.ntherm, mcc.nmeas, mcc.nsa, seed)

chseed (generic function with 1 method)

In [40]:
struct SSEBin{TPS<:Tuple}
    binsize::Int
    data::Vector{Vector}
end
SSEBin(size::Int, types::Type...) = SSEBin{types}([tp[] for tp in types])
function push!(bin::SSEBin, item)
    for (ii, vec) in zip(item, bin.data)
        push!(vec, ii)
    end
    bin
end

filled(bin::SSEBin) = length(bin.data[1])%bin.binsize == 0
function Base.show(io::IO, bin::SSEBin)  # only show the mean of last batch
    print(io, [mean(v[end-bin.binsize:end]) for v in bin.data])
end

In [52]:
types = (Int, Float64)
SSEBin{TupleFloat64, Int32)}(50, [[]])

LoadError: syntax: missing comma or } in argument list

In [29]:
function sweep!(model::AbstractModel, spins::SSESpins)
    lt = model.lattice
    NN = nsite(lt)
    NTAU = ntau(spins)
    beta = 1/model.temp
    dtau = beta/NTAU
    
    J_spatial = model.J0/NTAU
    J_Trotter = - log(tanh(dtau*model.Γ)) * model.temp / 2
    
    # visit each spin on the space-time lattice in sequential order 
    for itau = 1:NTAU
        for ir = 1:NN
            # interaction energy with the neighbouring spins in real space and 
            # the neighbouring spins in imaginary time 
            exchange_field = model.J0 * sum(view(spins, neighbors(lt, ir),itau))
            # periodic boundary conditions in imaginary time 
            itau_up = mod(itau, NTAU) + 1
            itau_down = mod(itau-2, NTAU) + 1

            exchange_field += J_Trotter * ( spins[ir,itau_up] + spins[ir,itau_down])
            energy_diff =  2 * spins[ir,itau] * exchange_field   # Note: J_spatial > 0 corresponds fo FM interactions 
            # Metropolis-Hastings update: flip the spin with probability min(1, exp(-beta*energy_diff))
            energy_diff < 0 || rand() < exp(-beta*energy_diff) && (spins[ir, itau] = -spins[ir, itau])
        end
    end
    spins
end

sweep! (generic function with 1 method)

In [32]:
sweep!(model, spins);

In [34]:
"""
measure observables

Returns a tuple of (E, Mz, Mz², Mz⁴).
"""
function measure(model::AbstractModel, spins::SSESpins)
    lt = model.lattice
    NN = nsite(lt)
    NTAU = ntau(spins)
    beta = 1/model.temp
    dtau = beta/NTAU
    
    J_spatial = model.J0/NTAU
    J_Trotter = - log(tanh(dtau*model.Γ)) * model.temp / 2
    # Measure the total energy 
    energy_tot = 0.0
    magnz = 0.0
    for itau = 1:NTAU
        for ir = 1:NN
            magnz += spins[ir,itau]
            energy_tot -= J_spatial * sum(spins[ir,itau] .* spins[neighbors(ir), itau]) / 2  # compenstate for double counting of bonds 
            # periodic boundary conditions in imaginary time 
            itau_up = mod(itau, NTAU) + 1
            energy_tot = energy_tot - J_Trotter * spins[ir,itau] * spins[ir,itau_up]
        end
    end
    magnz /= (NTAU * NN)
    energy_tot / (NTAU * NN), magnz, magnz^2, magnz^4
end

measure (generic function with 1 method)

In [45]:
function run(ntherm::Int, nmeas::Int, model::AbstractModel, spins::SSESpins, bin::SSEBin)
    println("thermalizing ...")
    for i=1:ntherm
        sweep!(model, spins)
    end
    
    println("measuring ...")
    for imeas = 1:nmeas
        sweep!(model, spins)
        res = measure(model, spins)
        push!(bin, res)
        
        println("E  Mz  Mz²  Mz⁴")
        if fitted(bin)
            println(bin)
        end
    end
    bin
end

run (generic function with 2 methods)

In [46]:
Random.seed!(2)

N = 27

bin = SSEBin(200, Float64, Float64, Float64, Float64)
lt = Chain(N)
model = TFIModel(1.0, 0.5, 0.1, lt)
initial_spins = ssespins(N, 200)

run(mcc, model, initial_spins)

TypeError: TypeError: in Type, in parameter, expected Type, got NTuple{4,DataType}

In [47]:
SSEBin(200, Float64, Float64, Float64, Float64)

TypeError: TypeError: in Type, in parameter, expected Type, got NTuple{4,DataType}